In [3]:
using DrWatson
@quickactivate "BNP2"
using Random, ProgressMeter, WeightsAndBiasLogger, MLDataUtils, BSON, Flux, Zygote
using MLToolkit.Neural, MLToolkit.DistributionsX
using Flux: Optimise
using Revise, BNP2

┌ Info: Precompiling MLToolkit [519e820e-097c-11e9-2274-1b004aeb0b9b]
└ @ Base loading.jl:1273
┌ Info: Precompiling BNP2 [91a68366-b384-5123-a8f1-3305599cf021]
└ @ Base loading.jl:1273


In [4]:
args = (
    dt          = 1f-1,
    n_trajs     = 10,
    is_noisyobs = false,
    σ_obs       = 1f-1,
)

;

In [5]:
function sim_traj(ms, qs, ps, dt, T)
    env = Space(Particle.(ms, qs, ps))
    traj = simulate(env, DiffEqSimulator(dt), T)
    return [env, traj...]
end

data_raw = 
let ms = fill(5e10, 3),
    qs = [[-1,  0],[ 1,  0], [ 0, √3]],
    ps = [[cos(π/3), -sin(π/3)], [cos(π/3),  sin(π/3)], [cos(π/1),  sin(π/1)]],
    T = 50, n_directions = 10, n_moving = 10, n_speed = 10, n_initials = 10, n_tests = div(3, 2)
    
    trajs, trajs_test = [], []
    # Trajectories with different initial conditions
    for i in 1:n_initials+n_tests
        traj = sim_traj(ms, add_gaussiannoise(qs, 1f-2), add_gaussiannoise(ps, 1f-2), args.dt, T)
        push!(i <= n_initials ? trajs : trajs_test, traj)
    end
    # Trajectories with different initial directions
    for i in 1:n_directions+n_tests
        traj = sim_traj(ms, qs, ps .* (1 + rand()), args.dt, T)
        push!(i <= n_directions ? trajs : trajs_test, traj)
    end
    # Trajectories with different speed
    for i in 1:n_speed+n_tests
        traj = sim_traj(ms, qs, rotate.(rand() * π, ps), args.dt, T)
        push!(i <= n_speed ? trajs : trajs_test, traj)
    end
    # Trajectories with different moving directions
    for i in 1:n_moving+n_tests
        traj = sim_traj(ms, qs, ps .+ [randn(2)], args.dt, T)
        push!(i <= n_moving ? trajs : trajs_test, traj)
    end
    (trajs=trajs, trajs_test=trajs_test)
end

for traj in data_raw.trajs_test[1:end]
    HTML(animof(traj).to_html5_video()) |> display
end

;

HTML{String}("<video width=\"500\" height=\"500\" controls autoplay loop>\n  <source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAA1421kYXQAAAKvBgX//6vcRem9\n5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTQyIHIyNDk1IDZhMzAxYjYgLSBILjI2NC9NUEVHLTQg\nQVZDIGNvZGVjIC0gQ29weWxlZnQgMjAwMy0yMDE0IC0gaHR0cDovL3d3dy52aWRlb2xhbi5vcmcv\neDI2NC5odG1sIC0gb3B0aW9uczogY2FiYWM9MSByZWY9MyBkZWJsb2NrPTE6MDowIGFuYWx5c2U9\nMHgzOjB4MTEzIG1lPWhleCBzdWJtZT03IHBzeT0xIHBzeV9yZD0xLjAwOjAuMDAgbWl4ZWRfcmVm\nPTEgbWVfcmFuZ2U9MTYgY2hyb21hX21lPTEgdHJlbGxpcz0xIDh4OGRjdD0xIGNxbT0wIGRlYWR6\nb25lPTIxLDExIGZhc3RfcHNraXA9MSBjaHJvbWFfcXBfb2Zmc2V0PS0yIHRocmVhZHM9NDggbG9v\na2FoZWFkX3RocmVhZHM9MyBzbGljZWRfdGhyZWFkcz0wIG5yPTAgZGVjaW1hdGU9MSBpbnRlcmxh\nY2VkPTAgYmx1cmF5X2NvbXBhdD0wIGNvbnN0cmFpbmVkX2ludHJhPTAgYmZyYW1lcz0zIGJfcHly\nYW1pZD0yIGJfYWRhcHQ9MSBiX2JpYXM9MCBkaXJlY3Q9MSB3ZWlnaHRiPTEgb3Blbl9nb3A9MCB3\nZWlnaHRwPTIga2V5aW50PTI1MCBrZXlpbnRfbWluPTIwIHNjZW5lY3V0PTQwIGludHJhX3JlZnJl\nc2g9MCByY19sb29rYWhlYWQ9NDAgcmM9Y3JmIG1idHJlZT0xIGNyZj0yMy4wIHFjb21wPTAuNjAg\ncXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAAQV2WI\nhAA///73aJ8Cm1pDeoDklcUl20+B/6tncHyP6QMAAAMAAAMAJAL8x7+hB0M0IAAAT0AEXpz+4Ovu\nAKQcEs1p7l+Q93qpHumsiRc7Jh7WLhQk5qKutEEDhRs+WR1Ax6JeC5az/ZrS9l5gRgDBhRq6reIo\nlwfV2JbqGvfegx9qW7n3TDwB5TINRX2NqdqWOCWfB3PB3B254TZEZw1q3e5BjRih1KZoLzakiq9A\n8rWja7DpCmflSvnAIl8w26pM+6FQ3rK98nTQi6YlqsRiJdseFFTYiWnrDzeViwMf/DAK26wMNweK\nEbK1HJpkxSAI6y4sAdv9B1UREc3+qP+8unSpY1sAOfvW+JMkBh3ikFstVo2+AQKrcyFHdDgOIGm/\nN95hk/7QaG5/YIkuAwKtGZIIjxZz7hHNE8vV9CfTrpwxWw7Zm2VDtgBF1FatIBnnMMIhREx2tSCE\npRIrVQqPyDZc38Sb7obd4KM7Cu6mH/NebnTBE16RC2jkc3tBh1NnCjoRs5D+xip8nvUq/MJsFU2W\nqRu8Wr5iNfgKHH+CasB4/dGSJ7+vzpk2pdIa+QxcP4CGp+yYgRP5XAOef8ZK5m9CdkcnhwXgW//h\nVuRxLR1uuvw4+ZKygQmSHIG5o8jdh7197ba//EtVX0UElBrzG5B1VsbxHTafeZMATndek4JD+EAO\nn295LhAZA/dER6Zs+hhr/M4hB8rcj4dNsQEKuoEg9GKAAAADABm8xdQM2BSl2SycF/LVAGG861PD\nbMJACb3Ygymu7NzzxIpu8jcD3LRkn/pOLL/hr38QJRFVlbj1o2ijy8D8wlE/sAcbRYZHBPMZOTUQ\n3TjZbqArQDFe8o1jhkyXyDCdBWCrgQ6DP+M/WEW9n6bIVY3b5yow9XGc4liZEg90graQWku9FPIl\ndTm2Ts1Vl9IPxqS9kEdqu3/pSrnvpRs8f+OmiKC9liJJKvYjN0f2mV/t2YgwFMomMjDBX7Ij0P0w\nRJGDehSfj+Sug7JjOQK3NNnyqepEYMz8v1uB+4BkXFff8BSUQ10wlTXJlxo20mXp3Nb4asiXbiJc\nPxbl+LH6zYnKiWfFL5qZJUanlOza+/ySf5lLXf/y+u1fIPMawEdVHJs5VswaDu4Ras0WLzJYXrZx\nuyQtMsqGa+NVxO9AIkNpfCkTyg0XFzu3OKejrW9dywwqB1+Gedi4IoD8yxlchQBfnY/btCmiP+SI\n83wihzMm3x9Y/hILSPzv9LZCyjX9kiHVbsr/jO3qkUGq7zT1vrmCMw4U513aadnEaLWFz7WE5/hI\nqR1VbrD9HQ/+Sxcdiz3Y0Y6da5k4MPEaXiDOzW1HNFhKD+bQFWtfEjcY+jaL77T8cJpKdz+gbIKh\nvaKS/tYE5AH+/33dM3xIjTm60zOKdYY+mXw8p7VVM/4efLuYRK4YAASlXVaMloGRtIACVcOVOx1g\nABoXLr2kP3godQ+23FTiaz07Sus4qxh+F68L9ulHP6+nXb6xmrOrlP7lSk9AHsVD365lh3iUGqzu\n/LZNtgteOYTfJc3toS8hA3bMpwTl/m2DE9r0n/eqpyscRtz0HTbLaiyXAmTQYj33ZIRqdmxosrRw\n9Vjkn+hiNkP9dfSlZeR/n3pNrG8+WQ8rRjfvufBW19hLTLAkqCqRM+vpxoiGZDVV58HyOvW19PkP\nLhlX+Hj2YfgB2TKYkFA9gNCrcl8Ob+sODlkGM2IiH1wLIsQdPLVR7+E9bCvxldVx4ZS9anoAFCCQ\nlyLIRSUashAQz3/K6bCafydFtLmpn5+mmbGsqeNPMFOR7fgjMCLcQQ+gCNZ2HAX5nxE6ILrOexoJ\n7hPh31IJHredZ5YWm98LSG9SKFI1MCCnknI6kXAVP9GtM+WSjIyO+Jz3Gd0uk4CwAc051WQc87nZ\nFi3fJbv7O9s/1yb0oyIwX2wrIqwlCfLGtGjUNtQY+tpqRDsZpd26gw1OD769AABDKVo/owaqYYVv\ngj/yUt/TfAZN2eCewabSbKsVgtMjHMDKteAtQTkvWW3DZ4ypHaKQkbFs9VCqW5Qu6+ryMCXCPkLf\nt0tY0MS8U/NefV3PhATY7Yvzyhid0iD+zaYqWuq7o9dqZ2Mv2ZHEtGGojYPfCRiQPl3iSlJDKnMy\nCbhRlNCpG9Ae+BBMV1cP9iY6hW91HOTi99qmB10GabSnTSTdzbNAkeYmxlqqVCbOOkx+yfV6zBrh\nh2De/M/JRU1g49NPA+UGFDfiAxdfO8mH/Vl+AbFnKvi7PiQci46u4JzTuPxSmBts8OqovtsI+P5y\nr7rFVxyiO6V4Z/5cvA06ACb9+cyzpwWTVfY1hJBFlvo1n230b982+dCKUrUksux/xseLXfAV+8sJ\nJASKKhk7GHPHaadgI7CK9q+GEqaUWfm1GjlG+YnfH29vZVHmUo/jd6wCDmXOATIMbeplXKXA2WSs\nNDw7Jy2w/JvoYLpPxcK+FhPeA8RjSoOR5p+y1kO+/R0gZpncH1AHxR/Xp8g2KMSzsXwobOHKDmlE\nWsu9ipzwaZo7APVhIOPS0/blrZZQNRSWnUbR9LJOdGRkMYPP5JtiaWBa8AAAre91ABNoWsnD7D57\n7sX57FOrhK0zNtZABgdAszilMaQiCz3nlptYzvqHZTEfmwQLA4IPcl2g8YAADNAXEOMaoW4xcdnN\nVa4TlLudKVKu3IDzajqjUoNYlCeMjw1FnLc2NzR03QjjJkQxGn2+lFPWSmREKNp1INXioOZH2B2a\nbI9hzasrk7zE9gMPTQs+J4ZB5kpCW0Q5QIHhlgBFP3Dq8Z

HTML{String}("<video width=\"500\" height=\"500\" controls autoplay loop>\n  <source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAA4OG1kYXQAAAKvBgX//6vcRem9\n5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTQyIHIyNDk1IDZhMzAxYjYgLSBILjI2NC9NUEVHLTQg\nQVZDIGNvZGVjIC0gQ29weWxlZnQgMjAwMy0yMDE0IC0gaHR0cDovL3d3dy52aWRlb2xhbi5vcmcv\neDI2NC5odG1sIC0gb3B0aW9uczogY2FiYWM9MSByZWY9MyBkZWJsb2NrPTE6MDowIGFuYWx5c2U9\nMHgzOjB4MTEzIG1lPWhleCBzdWJtZT03IHBzeT0xIHBzeV9yZD0xLjAwOjAuMDAgbWl4ZWRfcmVm\nPTEgbWVfcmFuZ2U9MTYgY2hyb21hX21lPTEgdHJlbGxpcz0xIDh4OGRjdD0xIGNxbT0wIGRlYWR6\nb25lPTIxLDExIGZhc3RfcHNraXA9MSBjaHJvbWFfcXBfb2Zmc2V0PS0yIHRocmVhZHM9NDggbG9v\na2FoZWFkX3RocmVhZHM9MyBzbGljZWRfdGhyZWFkcz0wIG5yPTAgZGVjaW1hdGU9MSBpbnRlcmxh\nY2VkPTAgYmx1cmF5X2NvbXBhdD0wIGNvbnN0cmFpbmVkX2ludHJhPTAgYmZyYW1lcz0zIGJfcHly\nYW1pZD0yIGJfYWRhcHQ9MSBiX2JpYXM9MCBkaXJlY3Q9MSB3ZWlnaHRiPTEgb3Blbl9nb3A9MCB3\nZWlnaHRwPTIga2V5aW50PTI1MCBrZXlpbnRfbWluPTIwIHNjZW5lY3V0PTQwIGludHJhX3JlZnJl\nc2g9MCByY19sb29rYWhlYWQ9NDAgcmM9Y3JmIG1idHJlZT0xIGNyZj0yMy4wIHFjb21wPTAuNjAg\ncXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAAKeWWI\nhAA3//728P4FNjuY0JcRzeidMx+/Fbi6NDe9zgAAAwAAAwAO4+Whwb68chzQAABbAAZZt/OESPzF\nAQDc9v+/3kXG2jTtX8hZA6Hbq6OtXw7r4y6BwSkKw89A81JE7XtlorEF7VC9D04OjSrBoAy/QwaD\nvIrtpyg7n1JDrcPIUvLtHSVgU3IDNZL4/RZe64FjS4JxmIA+CjPj1WSpr+nsvxz02a9vrBMciCVY\n08X425zHgv3RQ6KHdGlSQu/+yvCjyQhjUnA3/5CtyINLbf5eiPbEVRG4CsNzd1sxFy3tfdTy8u7i\n1fo0JkSy+6w3atFl/v9ABUuQPUpAlc5cNs20qUHXs7GBrhnv8Lq/uWBOz6v7Tva/f87AGDtlm2n0\nwHXahcbSLxGijo4E0Jr2Js+iH44gctUi8s+3X8WbyykrwFio/OvZQgDZXreM7oeN4nslvpvIicwf\nKQbdhWtQ1EPTj4nJvOIbFlgth8it/fl8rtC18MCynimHaGobKcgz9KcvztBvnYS/0iHlAFfT/rY/\nIJBOuDqXB7UXWRGK91PUwElGWtgFF8FT9FHK/l8A12Gswzln7pdCWNxPb9ZUvM71sfBzzJTf3TAx\nCjmOyfTPUgovk/hsDyX1kd2e09Y4DCyGGcQH3lP8hLS7A0BiXHnIVa1ODCnG0G1llZdctgvYuF4l\nBYm0ltkEQEo3Oc5CN7g8vZuOVdbkBV9R459Pas42xuVFH3UbbrcGXyo4eDZxRr9Cr6zxGGThgAPj\nPGTzjPze/Cv4Ldsa5mAzsbsM+Rgwd5aIphkz01PGLDtwRCe4kHCmEPKKTSFXK6E8UKWf/i4OWXcm\nIN1vHfRzyep3lSB+gx15zgBxEnEXkr0OEjsRKwvesGdVNam8b2tAYxfNoRG6cGAQrhDRsq2SmC9Y\nvqWCO5GxeLGm90gPLya6r6s3dDnXT/lf0J0HbHzZDyT5rf2f49oyZWCMvMFTC7ooZarBKCuFPZhO\njDgmhvcCeIAWvEEbswSWwAxElQHznA72UdaRdsxFfpuhCVck3C42QfFu3fxM8D14QEKNXDpSTnOG\nIh/VMauCf61sdLpULf7rjd4lkHU/xzzYs+tUuS/Bf4ST3MbcCEXBq5LUqOy7dRL8QxFgZdwX/Rqu\ngqMycbQX1Q7oQ6adYet/mVojN9bLmmuxX8/e+uGzulpjXmTqD9Cp7rPWmwZFDPBQK0Vcr2jGCH4y\nu88WPMZk+desQ2q2j6zlvSLXkMI1ADgZUbIiCV/upzuXMpDcW8hhYn1LtuDxkPF8AmdDTDJ/oHkM\nlz2UMpXyCPPeoP2nSyu/eHSl7LkxGouUIBXGWHl2HgEcdXLAwWas6K1b3werV8bniVywirZwoKlw\n+CV6tparRbP2yWKA6thOfJlfzeUc6Dfc9dWHrDdznUFPLTfp+tPsJWHdF2rRQQGgI5Gx4HW8mohb\n/s+87azYk4Ihy7yqYRWiSQETg/mxswwRMxg8KKsdwqdbPMoxunzX18ShhKbW55ZFmFqMVDXjwmbp\nQnvMATJQJ316I9G5H18OyDsdYcmPCdLPBQMEP7Bnid0W85HvZKMRT9JC2pG41MED53xndDG4yD14\nV6JHP1m+GW2+wd5LI/664A+PlYytomcVGeLrm40QRNGsLQt3b09Rmmt38CQo6MzKj/5xWurdy1GM\ngzPlwfDEi4szc4i+O9Ke2DqUEyY104vIIxOyVHmkLrJNcIHrJup4mW3LHKYKPaNObj/TFvCryayx\nWEWUqxCX896rwtvz3vxpBg+XmMdofBvhzyZWoZAPydejNWRWqHMfPnWVXVlOi86DNcVxQmLGnzNA\nihH86gAAJE4BMpqu6j4BKKDHVxvnsK/2eB1ZUu0AqiJ/fNsAgJ+KuAnRQUB1g1mG0sx65DU6cKsE\nDKeEn7yNDaNdRB+fhDsh3x7UJi1W9qqANs7+n+9xkRBk5tc675MCcFSJ6vsJSWJY8iSPKcq7qG2S\ntU4gVo4FGRY2XOc2zrqQggrPuRdiMMhVjWpkF9QQwlCgbrXdvIgI7rf2dy6Zvn5yAV9lz96LqcM1\nfYibYUXTdfVFAASEm2G1PhMBBTPT0If+7KZdtpstArbRSW3MP7EsmiZOnqWqx8eIWDoDOGyt/L/Y\nQTN0lzptzSL08WZNlnkdcpd0BbUd/ygYXf17nDcAKQZ733Qdx24v8xsDm7z64iVpqxvOSKafiCkD\nYussuUSRsb7MuGR+Elps/Lu31OKqhPWq541gCEvC0xLjNp/q/x67NH+4jH24kIQByp6sMUwfhFUJ\ncUQGND4GIMegY4PKSLsbcIKuiQJa0oD3rJqwFzWpnqID2JYtZw1THxhuSdxHXcMy+lkm3cRTjhVR\nmGAxh+TMlYIIr/zmXiSHHMh2evgke5XBHZ1VxpP8jv+JRahq7Mym6rfzWb6dt2imoojqAnHwLXKU\nh2YovZ3iYnDmTzJHxt2sB3zTa1QMEP2yOAsNHzSwtwNZxheLME4dL8fptcBNEdNknfZlXPMaWXSd\nS8dNbL08mHCT6+Iu/xQA2ZdK5P+9jk7o/Y2uC1k8INDmdID3a1B4SeBx+LzvYNMobTNOD6z779GO\nGxJi7tHYorzZ3pQAsL9xPs+yASSUIwkzssJQJVmm0FtL7JnUcnhZ8EpArrOahSacO/XaW7LfXGRc\nGODwlV8AcycxO/PuLSwtBc6fakmPDqXdF1u3H+gTYJB9Mi1oXrVD9aK6wPteACtfOIdz9woH5gVN\ncSjnNXgF851FDDQdEcp/rnlWfzXZ7S5eZ9uMMxOnsltp6M

HTML{String}("<video width=\"500\" height=\"500\" controls autoplay loop>\n  <source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAA79m1kYXQAAAKvBgX//6vcRem9\n5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTQyIHIyNDk1IDZhMzAxYjYgLSBILjI2NC9NUEVHLTQg\nQVZDIGNvZGVjIC0gQ29weWxlZnQgMjAwMy0yMDE0IC0gaHR0cDovL3d3dy52aWRlb2xhbi5vcmcv\neDI2NC5odG1sIC0gb3B0aW9uczogY2FiYWM9MSByZWY9MyBkZWJsb2NrPTE6MDowIGFuYWx5c2U9\nMHgzOjB4MTEzIG1lPWhleCBzdWJtZT03IHBzeT0xIHBzeV9yZD0xLjAwOjAuMDAgbWl4ZWRfcmVm\nPTEgbWVfcmFuZ2U9MTYgY2hyb21hX21lPTEgdHJlbGxpcz0xIDh4OGRjdD0xIGNxbT0wIGRlYWR6\nb25lPTIxLDExIGZhc3RfcHNraXA9MSBjaHJvbWFfcXBfb2Zmc2V0PS0yIHRocmVhZHM9NDggbG9v\na2FoZWFkX3RocmVhZHM9MyBzbGljZWRfdGhyZWFkcz0wIG5yPTAgZGVjaW1hdGU9MSBpbnRlcmxh\nY2VkPTAgYmx1cmF5X2NvbXBhdD0wIGNvbnN0cmFpbmVkX2ludHJhPTAgYmZyYW1lcz0zIGJfcHly\nYW1pZD0yIGJfYWRhcHQ9MSBiX2JpYXM9MCBkaXJlY3Q9MSB3ZWlnaHRiPTEgb3Blbl9nb3A9MCB3\nZWlnaHRwPTIga2V5aW50PTI1MCBrZXlpbnRfbWluPTIwIHNjZW5lY3V0PTQwIGludHJhX3JlZnJl\nc2g9MCByY19sb29rYWhlYWQ9NDAgcmM9Y3JmIG1idHJlZT0xIGNyZj0yMy4wIHFjb21wPTAuNjAg\ncXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAANNWWI\nhAA3//728P4FNjuY0JcRzeidMx+/Fbi6NDe9zgAAAwAAAwAO4+Whwb68chzQAABbAAZZt/OESPzF\nAQDc9v+/3mOI2WenSzhmXOZf5E/6SR3YzI2VrHCfw1iGpsvPPgaRBsRhwWZ4CgufVjmyMKqlY1rR\nvl62K0QV/qiNrBXnuCAer5kxwTkIY2m+4ve24ojd81PxTL0vJJyUKXPr+fTwcrzB8iyVKT5bsNp4\nMeoZdy70s+T/hbeYMUpqsMqighPdP+Sn9n+IyvExH2lDgqKkgxfz96QJrDlegOIDe4kCB1WvmY7k\nYPtJi1nyOdcN4jAFLvHtuzO6vvOxIRfIUxJWhKw7LB/LemPeJ8G7zG/3pEsyG3gSOnuUtxrsv/4y\nctSSbsp51VmzKArpzdiwvMnBegBb2FaljX11AAWaEDX4zMKZ5WgE4HL4fH/LUfjWE76FtK/rwTcd\nxrIsTwVynn03ltzvuiPIMNuDY4a1SW4VladfAmZJCsDxZPiLaErTRWL1hBsPwZLTBoNv+3FdFlGt\nxCVuXKUuPVPko7F4a9/oWif6T01YWVqbLd/dUs3EryKcDAdQYAvl/GDnyDz419b3AYe+0P6hd328\n6oyJa9QDTvkUfmUEcG7cOf7aOSFP9oYrNQt01CPgw0z4FDbigMPxNhj+4PMHzgYU2osszqxQxUHH\ng8GtmtdNcOsEzFowIm2p3WVXa90tEXfrq+HcU2PP8qvXqowU9NZG0n6k+djiLHm2pZS/7OSK4fRB\nH58ADeS7wi8B8kISnlc3JQrnX2lZc/x1yFGxKvyPhXLDbv9/XD/TdAJ6g6P5b/whIIiHY6O2thhH\nppOB3qICkiS0j0UmY9wlOiL36QJ/CMIinSX7b/MOgv1CK0zq0BL8LWybCB29qQAsh6qpRumtHbSN\nHfrujzq3/flUFu+EvnbnQ00zCtICcjm+ISyZk0kaE8qNqpNO8r539fjVZWOYbjleDvpehCmDougf\nz+xrwRptwcE8kFw9gjCaicMKO1KAR1H8hDz7gEaBtj68Hr/Z7G+2vOWuiNb9j7ReyQDeQ/aIf7Yb\nOLcPyXgHGVEyN/qWzU9IQJqDHRfF53jKQSRDiJrNqCFA2XAeYALtVcc50sBUfeesA3dQzrxDHGM6\nJknDkNJu88O2K0eyoZ730KAKEs/OHykOgUJTv3BSxZH2F7EU3QXLbqhKzTk+FImu1ZjPS/wIy+YO\naVtHNasPaskH5TCWrL6Pj21fI76GfSuH3Wr4vx/zc2CACPptykl6YCHhpH6vRTdxHsNZmZy32CM6\nWQ6i3V9GhKcU/xK3hUI7ZnVLqr8RzHjNoGHXZqtMABALM/D27T0CR6MYAEivU12D0GWXABWrdqG/\ntQkhPWWl71bmnLqEnn1M5KcPN/1kJ+jAt4CYN6KxGLjofvQh64P6gKxwjce9fJhQS+Cx+vFJDDW4\nWIT+ogplFIwa3WPJ9BpN9i6R+XXJfBbDXEzliH2G3KqNKAr4nhqOeShCkhivJpkwxGxlFHucN9Bz\nr9eTTZHZ6VytjR//030qwlGsFCH/Em4Yb/jx/m/qkR8TQQgQQPWb/MABWgLAIGpmYe4o48abOW/t\nUM3qnf4revtDOw7STjEDino90gJkJQjvI+JfmAqNWPkUn2KxKQh9swF/+umRt+JafAc4+74c7CR6\nWCPVlqPKXffAfzbvWp9VpkTzP1q7NartqklwWyApeRR+2nIrSEG2ZzJvwYxdgGWzLTVS5ql2V5BG\ng/VHPyLSq+KbDhwADIgI86SMg0QDLXHVqCDjcn0M3fY52bFTYkzB9t1bIPvmKXGGkGzXWlsTpoKb\nZE+RibXerQUpGK6M1Ddx2Ik/ZiZvgMwWYHVtc33mGKDQ1hmFYt0gw9xsQ1QaY2CZoFxXj+YSoOk0\n40eVZONznbSPl4zvMwFbiaBRFtBT/g7IvS4WlDaIB3drImdIJJbe+nLgwKtgSQsvOCBdOjbERhe+\nY6Mg7Nf70qeWnkmjx/Rhojv50FXyaymSoP/53lBia11U7iInSPsB73g+n6GOTirQKZb5WMVjM388\nkLAdzVlXihdkrDIYQfDCSH6j3rtkBJUgz5SxoYsgoM52OO5+Fm22AJKX4qSPi+IdexvPsekZi00+\n+Wtv5jk4nxRPzWrl8ZFPjLqbHv6wZ2Wkhgq8WrKVDzfEfemOx0lG9IoMvU48IwdW6bLX7H+jjXHS\nEBwBgrjqa++Vv9X8ExpMfKDxUiXzii5FbiCEybE8N8rtVN/pF9CfxN9i0LiD6sdj1+1YgkwfhkAX\nSwBRO7LecJfpGmZXDUIs1vmY0FuruUZt7EVgRwAN1BRSCVZI67z4nGcAYaV/1r52PG7LOXyfefUI\nmG+itsnR/KpNAkTlA6w72uvY9HHiInhs1bIGDcI7AX2VxYvjTNpkttp0dzJnpM5OBgOtPAo25lZH\n5xSZSVxTfQQ6maLJRpuwXn2iC5KkqdxPj9E1i687TphBgy/EOeNVZC3s6ZAsfLLQgv1R4yRg8Aq2\n/MZY0+fn917y+8wP9ZWtWEVMv1PKMrpMbSxSKhPZxm3/OvxDSls4gj34cguUWyEOLxzwDJPzXsAK\nlQeHzlr4cJbO2GsoD8UTLsw/6a5vBggBVKjHnHPfHFhCD7yyse1m1iDynRAaY5OBgCqCFxtPRDQ3\nyih9hmSiyUoNMyyutWJ/0umjARrguHcwt226yhDxbquUio0Lj1mlsCWNGfBhSDxtME6CAsuJ4PYR\nY0ZHoZvE1RSx0/SemamEqwzKxd51g0Y1O0Q/yc9/ul8UW4

HTML{String}("<video width=\"500\" height=\"500\" controls autoplay loop>\n  <source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAHGZ0eXBNNFYgAAACAGlzb21pc28yYXZjMQAAAAhmcmVlAAB/0G1kYXQAAAKvBgX//6vcRem9\n5tlIt5Ys2CDZI+7veDI2NCAtIGNvcmUgMTQyIHIyNDk1IDZhMzAxYjYgLSBILjI2NC9NUEVHLTQg\nQVZDIGNvZGVjIC0gQ29weWxlZnQgMjAwMy0yMDE0IC0gaHR0cDovL3d3dy52aWRlb2xhbi5vcmcv\neDI2NC5odG1sIC0gb3B0aW9uczogY2FiYWM9MSByZWY9MyBkZWJsb2NrPTE6MDowIGFuYWx5c2U9\nMHgzOjB4MTEzIG1lPWhleCBzdWJtZT03IHBzeT0xIHBzeV9yZD0xLjAwOjAuMDAgbWl4ZWRfcmVm\nPTEgbWVfcmFuZ2U9MTYgY2hyb21hX21lPTEgdHJlbGxpcz0xIDh4OGRjdD0xIGNxbT0wIGRlYWR6\nb25lPTIxLDExIGZhc3RfcHNraXA9MSBjaHJvbWFfcXBfb2Zmc2V0PS0yIHRocmVhZHM9NDggbG9v\na2FoZWFkX3RocmVhZHM9MyBzbGljZWRfdGhyZWFkcz0wIG5yPTAgZGVjaW1hdGU9MSBpbnRlcmxh\nY2VkPTAgYmx1cmF5X2NvbXBhdD0wIGNvbnN0cmFpbmVkX2ludHJhPTAgYmZyYW1lcz0zIGJfcHly\nYW1pZD0yIGJfYWRhcHQ9MSBiX2JpYXM9MCBkaXJlY3Q9MSB3ZWlnaHRiPTEgb3Blbl9nb3A9MCB3\nZWlnaHRwPTIga2V5aW50PTI1MCBrZXlpbnRfbWluPTIwIHNjZW5lY3V0PTQwIGludHJhX3JlZnJl\nc2g9MCByY19sb29rYWhlYWQ9NDAgcmM9Y3JmIG1idHJlZT0xIGNyZj0yMy4wIHFjb21wPTAuNjAg\ncXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAAJoWWI\nhAA3//728P4FNjuY0JcRzeidMx+/Fbi6NDe9zgAAAwAAAwAO4+Whwb68chzQAABbAAZZt/OESPzA\nwgHpubiDcPuQlInrDyjfe4sZLEXz7GLA1scCPeAQOG2vpWyi72obqBPBl0kMR+aMsSV6cEKOwDV/\nXV6VklcN32Ieer/Y2XgSJFFvY1RplvJbCIkUBt08qTUOvzHAx3eVd97q5ZHXAoAmshej0B6EDlRj\ntfhIgUqfWxfXdB82cfRYETKnsFMzR3Q8BlQAINwBdE81y2w4D6W0tQhooYPJL6AqWPAKGl3dRqK9\nE4ZireuGnh6yCMVyc4FQU1tVvuRGBeuti3ceZZiPb5jA/7Ay1BGg+yxpgkFW5m6H7bB9vpEGsQGP\nT0MQ7DFjoK0m2vwASXckejW/eUxtb9N8sqhPq+KQ4DdWg/5jY9z9SZt5z4OBLKjma5e336A6m9yy\nrejMDVM09FY+fHJ+gtlRJaySN5NyrMUT0kHxjPa7FL3m/YCiYrAVFpk5mj8/lp5+WNAcOfrGJ73i\nYAAABk4UpNkjbA96k87FuaVS89eCMmO9A6V2SZu43VAVI3O2SrX8be8BHNDhnGNMxuxQZ6XmXx/e\nJMu0agOxQyWXgGMKwHEBm2ifAmSG+dEvUrW6v0RZvEgfcRlVT9JLG8SjGd6iSD2vqQkXgX5cw9CO\nIWbAx3aV+MdDGMF0skyDolSNgj2wo9XQqlyQDHCnDm8LmaXgu96J4uJNlm8EgLVK8epPo/17YgYn\noArRFMMmemp4xYeBJ8UgiaYYGNKXQ99fK846hSz/8XByy7kxBZB6g5Ixadyvrsl20gvjFMK2nvnW\ntNkty2xNVOHCm15DU8gK6Pvn0lU7p3AyjbmBxQYgY7CL2qNZKNv3LA5TPYjiCZwjaEjbwR0H1zqn\nKIYflvtNcPyBBnfrSPeeff2eX7I/wqXCYn6cj4Kr9HKdzzLWz2yJcPZ7XZ/3lgAAL47B/qPfALqJ\nDToe+ThKYSCh7vCm+69yhoIjgShNf70/Wss0CTgulAB2qdBsnfzRtOBTGSXXmSs761T934sfz1Ir\nxwW2t1FcPRx0Xp1NSpU0TFu0G0suHxHOXh/aJSsynOWff7l5YRmPtmzTj1a489aedO2Nx7CJYrEW\nQPOCO0+cvsQJJ+byiTMX8imwN6EU1lUM594RgNvfc9VdbL3uqoeD5NnBqvvpOTleIh6DuPX8o4ZA\nAABlwCpamCbyWsl+vtUpSse+mbJYXlrAQMJG3ghNOHkbKinnZxqvQ8pIxqtYeyctkbR/8gnFLwjx\nz/cC12M3m5Rs+rJLY2LVzn/kSQaiVv7RSYZQxgA5BKnw+De6Ljc292RtBosOwM48xeU33F4yj09L\nicJ3euBwiLKkkUp3bKutYipcOsXuiDByndkyn3J+wEi77N0bkP2QI1hg4bw7W9t2pUPP5MSX7Bob\n4boSgvZa2dQKo9I7/RbmJ72JDOdHxQl+1nCPYWSHGSiDQSOQ9uTkT9ejjADzjpomA6Y9m7YS7AfD\nx+d6ybDal1F7xPfAo0LD1p78U7JrJKHZRtyQPWb9mj4ZKN8VrmChx6BEbzGbqshjnxe1QcT4RxQY\nnyiElD2DSkLoZjOSjSo6QtMafrz8Dad2HgXQ8/a4LkMfddk5hKXhN6Jx4YyNUsE2gEHRiTBbDHAY\nybAQoDrXNtmkabp5oEv46QUuZTViZTeyboTJh6KLk5zyieNijkC0DdhiGkAehgk+7GRM6qlYrdpw\nCKSpiCixEkwbNGapFyK1sj+3YsMki+gAAb45o5sdYKmVOAkrdrCmPafkh95VPJtOWw/Qv3oaCHur\noUqyzsjvEtVcZTAUGnqY9lFT6S1UClf1HrysVBSMRBgFlewhXpMHiJYwvzUj1jpRegKKkHeqLIQI\nTI58aBMbtOrCyJMgsW6l2NpbA0IsMF0MEZTsC0zx88dZSsKpTyaWyzhuhK9rIKehbLiaFhkjcU8P\neKVV133UItB/7BVgj3HWfqSnZwPaLKowYvbOGbvqdPZVo9rRfxEdmyns8G8uDZfZFUaz+eMTKGZF\ngIuXLawG1QAAGy7xJm49X1KrFUWcNa1oyT8QAKh1S0AAAAXftcFGjrD9A7zWzlsjkQxC3OEvdg0Y\nzOUBqWa3cCycBUQdiSMtfwsORE45FgjNbs9m7aQ+Ov/vjVe+wr1kdPk+o32dC9zbQOvcl7BCSABH\nxyERf1rjWrec84TRhZC62eTZz7sGp3aLbpXf8ewhMHnikFvrum5VUv3plE4FDAkdRpnpjF0dHmHR\nWiRmeh07BSqFOA3m5ujpla+mRFyV9IpU8B2p5f0RGEr42qPHw60kQ2ysqW07Z1v5SUpG3bguHY5R\nd0Y0c2Df//I9FzL1stZfHyFt7ZOiK9RztXU134VtGppS0srpXPsSAnrPniidEwn9EMi/mPGHsGS5\ncr8usWhYOoVXZh6QqhE1C8eESPeZQuoFcju/34Jfg65CtpKBUJ1sa1fDLwWx5xhUlueKU1fL1QDK\nR+QwgqAoEUWg+j/a3j3xycqTbhwcHzotgiqckzajD764+88LSzHLyXd0tGWI/MyZdU4lUlESGqfl\nxQO/1MatF9+u7qUCqsF6/EEjJvWwyHQjG+mO7Nco2g8fLM2Eoz6IJnLFmhRU8ph5I8oDiIs9vKyM\neSWsLW0LrbtLk1poBY29wTkAC9bHmxUSiNATNL2cQqbFuJ9gcBPap7ap2gKhNJuYSJVIaYehng7q\nIlZ30EMR9DBUQUYKjQ75E+IJUGj/PUnqZQZ3Vckg030/0o

In [6]:
function preprocess(data; do_shuffle=true)
    # Convert trajectories to training pairs
    s_list, s′_list = [], []
    for traj in data.trajs
        states = stateof.(traj)  # Vec{Mat}
        states = vec.(states)    # Vec{Vec}
        states = hcat(states...) # Mat
        # Add Gaussian noise
        args.is_noisyobs && (states = states + args.σ_obs * randn(size(states)))
        # Create pairs of x_{t} -> x_{t+1}
        s, s′ = states[:,1:end-1], states[:,2:end]
        push!.((s_list, s′_list), (s, s′))
    end
    s, s′ = Matrix{Float32}(hcat(s_list...)), Matrix{Float32}(hcat(s′_list...))
    return (s=s, s′=s′)
end

data = preprocess(data_raw)

@info "Processed data" length(data_raw.trajs) length(data_raw.trajs_test) size(data.s) size(data.s′)

;

┌ Info: Processed data
│   length(data_raw.trajs) = 40
│   length(data_raw.trajs_test) = 4
│   size(data.s) = (12, 2000)
│   size(data.s′) = (12, 2000)
└ @ Main In[6]:20


In [47]:
gpu(data[:,1:5])

(s = Float32[-0.9947667 -0.9394256 … -0.7936965 -0.7044908; -0.013464166 -0.09409873 … -0.22922291 -0.28129536; … ; -1.0049078 -0.99859375 … -0.94751054 -0.9008759; -0.010137976 -0.15378906 … -0.44318953 -0.58958435], s′ = Float32[-0.9394256 -0.87220806 … -0.7044908 -0.6052438; -0.09409873 -0.16640484 … -0.28129536 -0.32122526; … ; -0.99859375 -0.9796907 … -0.9008759 -0.83802325; -0.15378906 -0.2980196 … -0.58958435 -0.7373483])

In [50]:
include(srcdir("Models.jl"))
import .Models

nf = Models.NeuralForce((50, 50), args.dt, args.σ_obs) |> gpu

Zygote.refresh()

let batch = gpu(data[:,1:5])
    Models.lossof(nf, batch...) |> display
#     @code_warntype Models.lossof(nf, batch...)
end

;

(loss = -12.492491f0, lp = 12.514783f0, l2 = 0.02229226f0)

In [51]:
logger = WBLogger(project=projectname(), notes="")
config!(logger, args)

Loading chipmunk for Linux (64bit) [/afs/inf.ed.ac.uk/user/s16/s1672897/miniconda2/envs/ml/lib/python3.6/site-packages/pymunk/libchipmunk.so]


wandb: Tracking run with wandb version 0.8.29
wandb: Run data is saved locally in wandb/run-20200307_173243-umckzx8r
wandb: Syncing run fallen-fire-68
wandb: ⭐️ View project at https://app.wandb.ai/xukai92/BNP2
wandb: 🚀 View run at https://app.wandb.ai/xukai92/BNP2/runs/umckzx8r
wandb: Run `wandb off` to turn off syncing.



In [55]:
let n_epochs = 1_000, opt = Optimise.ADAM(2f-4), ps = params(nf), batch_size=100
    with(logger) do
        @showprogress for epoch in 1:n_epochs
            data_shuffled = shuffle(data)
            for (iter, batch) in enumerate(eachbatch(values(data_shuffled); size=batch_size))
                batch = gpu(batch)
                local res
                gs = gradient(ps) do
                    res = Models.lossof(nf, batch...)
                    res.loss
                end
                @info "train" res...
                Optimise.update!(opt, ps, gs)
            end
        end
    end
end

;

Progress: 100%|█████████████████████████████████████████| Time: 0:05:53m


In [ ]:
let traj_true = data_raw.trajs_test[4], state = vec(stateof(first(traj_true))), T = 50
    function envfrom(state)
        dim = div(size(state, 1), 2)
        return Space(Particle.(fill(5e10, 3), state[1:dim], state[dim+1:end]))
    end
    traj = [envfrom(state)]
    for t in 1:T+20
        state = Models.leapfrog(nf, state)
        push!(traj, envfrom(state))
    end
    HTML(animof(traj).to_html5_video()) |> display
    traj_combined = [Space([objectsof(traj[t])..., objectsof(traj_true[t])...]) for t in 1:T+1]
    HTML(animof(traj_combined; refs=[4, 5, 6]).to_html5_video()) |> display
end

;

## MCMC

In [ ]:
@model three_body(ms, states, dt) = begin
    q ~ MvNormal(zeros(6), 10)
    p ~ MvNormal(zeros(6), 10)
    space = Space(Particle.(ms, q, p))
    for i in 1:length(states)
        space = transition(space, dt)
        states[i] ~ MvNormal(vec(stateof(space)), 1e-1)
    end
end

@time chn = sample(three_body(massof.(data.objs0), data.states, args.dt), args.alg, args.n_samples)

splot(chn; colordim=:parameter) |> display

bson(datadir("three_body-noise=$(args.σ).bson"), chn=chn)

;

let n_mc = 50, T = 100, do_mean = true, res = get(chn[end-n_mc+1:end], [:q, :p])
    Q = Matrix{Float64}(hcat(res.q...)')
    P = Matrix{Float64}(hcat(res.p...)')
    if do_mean
        Q = mean(Q; dims=2)
        P = mean(P; dims=2)
    end
    Q̂ = zeros(6, T)
    for i in 1:size(Q, 2)
        q, p = Q[:,i], P[:,i]
        space = Space(Particle.(data.ms, q, p))
        traj = simulate(space, args.dt, T)
        Q̂ = Q̂ + hcat(positionof.(traj)...)
    end
    Q̂ = Q̂ / size(Q, 2)
    HTML(animof(Q̂).to_html5_video()) |> display
end

;